In [1]:
%%capture
!pip install -qU langchain-groq

In [2]:
import os
from sam_sk import api_key

os.environ["GROQ_API_KEY"] = api_key



In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages =[
    SystemMessage(content = "English to Italian Translator!"),
    HumanMessage(content = "hi!"),
    ]

model.invoke(messages)


AIMessage(content='Ciao! Come stai oggi? (Hello! How are you today?)', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 22, 'total_tokens': 39, 'completion_time': 0.014166667, 'prompt_time': 0.002755013, 'queue_time': 0.011756336, 'total_time': 0.01692168}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-1fb68411-1178-4955-b1fd-96c00eb063e1-0', usage_metadata={'input_tokens': 22, 'output_tokens': 17, 'total_tokens': 39})

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
result = model.invoke(messages)
parser.invoke(result)

'Ciao!'

#### The `|` operator is used in LangChain to combine two elements together.

* Chain the model with output

In [6]:
chain = model | parser
chain.invoke(messages)

'Ciao!'

we can see that the chain has two steps: first the language model is called, then the result of that is passed to the output parser

## Prompt Templates

LangChain offers a tool called PromptTemplates that converts the users raw input in such a way model can understand

Let's create a PromptTemplate here. It will take in two user variables:

* `language:` The language to translate text into

* `text:` The text to translate

In [7]:
from langchain_core.prompts import ChatPromptTemplate

#lets create a string that we will format to be the system msg
system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", "{text}"),
    ]
)

In [8]:
result = prompt_template.invoke({"language": "Italian", "text":"Hi, Beautiful!"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi, Beautiful!', additional_kwargs={}, response_metadata={})])

We can see that it returns a `ChatPromptValue` that consists of two messages. If we want to access the messages directly we do:

In [9]:
result.to_messages()

[SystemMessage(content='Translate the following into Italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, Beautiful!', additional_kwargs={}, response_metadata={})]

# Chaining together components with LCEL

In [10]:
chain = prompt_template | model | parser
chain.invoke({"language":"Italian", "text":"Hi, Beautiful!"})

'Ciao, Bella!'

# Serving with LangServe

Now that we've built an application, we need to serve it. That's where LangServe comes in.
